# Proyecto Riesgos en Entidades Financieras
## Estudiantes: Ingrid Fonseca, Jimena Chacón, María José Bolaños y Sofía Sequeira

## Librerías

In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
# Lista de ISIN y fechas de adquisición
isin_data = {
    'US30303M1027': '2021-12-29',
    'US8969452015': '2021-12-28',
    'ES0177542018': '2021-12-29',
    'IT0003128367': '2021-12-30',
    'ES0130960018': '2021-12-30',
    'ES0130670112': '2022-01-04',
    'US2546871060': '2022-02-10'
}

In [13]:
# ISIN mapping to ticker symbols for yfinance
isin_to_ticker = {
    'US30303M1027': 'META',
    'US8969452015': 'TRIP',
    'ES0177542018': 'IAG.L',
    'IT0003128367': 'ENEL.MI',
    'ES0130960018': 'ENG.MC',
    'ES0130670112': 'ELE.MC',
    'US2546871060': 'DIS'
}

In [14]:
# Monedas de los tickers
isin_to_currency = {
    'US30303M1027': 'USD',
    'US8969452015': 'USD',
    'ES0177542018': 'EUR',
    'IT0003128367': 'EUR',
    'ES0130960018': 'EUR',
    'ES0130670112': 'EUR',
    'US2546871060': 'USD'
}

In [15]:
start_date = max(isin_data.values())

In [16]:
# Descargar datos de precios desde Yahoo Finance
precio_historico_usd = {}
precio_historico_eur = {}

for isin, fecha in isin_data.items():
    ticker = isin_to_ticker[isin]
    try:
        data = yf.download(ticker, start=start_date, end='2024-08-01')
        if not data.empty:
            if isin_to_currency[isin] == 'USD':
                precio_historico_usd[ticker] = data['Adj Close']  # Precios ajustados en USD
            elif isin_to_currency[isin] == 'EUR':
                precio_historico_eur[ticker] = data['Adj Close']  # Precios ajustados en EUR
        else:
            print(f"No se encontraron datos para {ticker}")
    except Exception as e:
        print(f"Error al descargar datos para {ticker}: {e}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [17]:
precio_usd_df = pd.concat(precio_historico_usd, axis=1)
precio_usd_df.columns = precio_historico_usd.keys()
precio_usd_df

,META,TRIP,DIS
Date,,,
2022-02-10 00:00:00+00:00,227.385376,29.340000,150.969223
2022-02-11 00:00:00+00:00,218.890945,28.610001,148.300262
2022-02-14 00:00:00+00:00,217.046494,28.370001,149.669464
2022-02-15 00:00:00+00:00,220.336594,30.070000,153.509171
2022-02-16 00:00:00+00:00,215.889969,29.230000,155.126434
...,...,...,...
2024-07-25 00:00:00+00:00,452.977875,17.610001,89.209999
2024-07-26 00:00:00+00:00,465.256165,17.900000,89.930000
2024-07-29 00:00:00+00:00,465.266144,17.969999,92.139999


In [18]:
precio_eur_df = pd.concat(precio_historico_eur, axis=1)
precio_eur_df.columns = precio_historico_eur.keys()
precio_eur_df

,IAG.L,ENEL.MI,ENG.MC,ELE.MC
Date,,,,
2022-02-10 00:00:00+00:00,175.426514,5.415340,14.436003,15.766636
2022-02-11 00:00:00+00:00,171.825500,5.383347,14.416932,15.649594
2022-02-14 00:00:00+00:00,162.144089,5.321043,14.188092,15.323561
2022-02-15 00:00:00+00:00,169.621597,5.412814,14.146139,15.490762
2022-02-16 00:00:00+00:00,170.585815,5.481011,14.165208,15.574358
...,...,...,...,...
2024-07-25 00:00:00+00:00,162.193283,6.660000,13.670000,18.040001
2024-07-26 00:00:00+00:00,164.456207,6.670000,13.620000,18.025000
2024-07-29 00:00:00+00:00,161.406174,6.650000,13.650000,18.020000


In [19]:
#Transformación de las acciones en euros a dolares
tipo_cambio_eur=pd.read_excel('tipo_cambio_eur.xlsx')
tipo_cambio_eur=tipo_cambio_eur.dropna(subset=['Tipo Cambio'])
tipo_cambio_eur2=tipo_cambio_eur.groupby(['Año', 'Mes']).agg(promedio_tipo_cambio=('Tipo Cambio', 'mean')).reset_index()

# Opcional: Crear una columna 'año_mes' en formato "YYYY-MM"
tipo_cambio_eur2['Año_Mes'] = tipo_cambio_eur2['Año'].astype(str) + '-' + tipo_cambio_eur2['Mes'].astype(str).str.zfill(2)

# Reordenamos columnas para que 'año_mes' y 'promedio_tipo_cambio' estén juntos
tipo_cambio_eur3 = tipo_cambio_eur2[['Año_Mes', 'promedio_tipo_cambio']]
tipo_cambio_eur3


,Año_Mes,promedio_tipo_cambio
0,2022-02,1.134155
1,2022-03,1.102483
2,2022-04,1.080676
3,2022-05,1.057945
4,2022-06,1.056709
5,2022-07,1.017643
6,2022-08,1.013091
7,2022-09,0.989377
8,2022-10,0.982838
9,2022-11,1.020482


In [20]:
precio_eur2=precio_eur_df.reset_index()
precio_eur2['Año'] = precio_eur2['Date'].dt.year
precio_eur2['Mes'] = precio_eur2['Date'].dt.month

# Ahora unimos df_precios con df_resultado (que contiene el promedio del tipo de cambio mensual)
df_combinado = pd.merge(
    precio_eur2,
    tipo_cambio_eur2,
    left_on=['Año', 'Mes'],
    right_on=['Año', 'Mes'],
    how='left'
)

precio_columns = [col for col in df_combinado.columns if col not in ['Date', 'Año', 'Mes', 'promedio_tipo_cambio','Año_Mes']]

# Creamos las columnas de precios en dólares para cada acción
for col in precio_columns:
    df_combinado[f'{col}_dolares'] = df_combinado[col] * df_combinado['promedio_tipo_cambio']

precio_dolares_columns = [f'{col}_dolares' for col in precio_columns]
df_dolares = df_combinado[['Date'] + precio_dolares_columns]

# Restauramos 'Date' como índice
df_dolares = df_dolares.set_index('Date')

# Mostramos el resultado
df_dolares

,IAG.L_dolares,ENEL.MI_dolares,ENG.MC_dolares,ELE.MC_dolares
Date,,,,
2022-02-10 00:00:00+00:00,198.960858,6.141835,16.372665,17.881809
2022-02-11 00:00:00+00:00,194.876751,6.105549,16.351036,17.749066
2022-02-14 00:00:00+00:00,183.896529,6.034888,16.091496,17.379293
2022-02-15 00:00:00+00:00,192.377183,6.138970,16.043914,17.568925
2022-02-16 00:00:00+00:00,193.470755,6.216316,16.065541,17.663736
...,...,...,...,...
2024-07-25 00:00:00+00:00,175.929644,7.224044,14.827730,19.567832
2024-07-26 00:00:00+00:00,178.384218,7.234891,14.773495,19.551560
2024-07-29 00:00:00+00:00,175.075873,7.213197,14.806036,19.546138


In [21]:
df_combinado['promedio_tipo_cambio']

,promedio_tipo_cambio
0,1.134155
1,1.134155
2,1.134155
3,1.134155
4,1.134155
...,...
629,1.084691
630,1.084691
631,1.084691
632,1.084691


In [22]:
cartera_usd= pd.concat([precio_usd_df,df_dolares],axis=1)
cartera_usd= cartera_usd.rename(columns=lambda col: col.replace('_dolares', '') if '_dolares' in col else col)
cartera_usd

,META,TRIP,DIS,IAG.L,ENEL.MI,ENG.MC,ELE.MC
Date,,,,,,,
2022-02-10 00:00:00+00:00,227.385376,29.340000,150.969223,198.960858,6.141835,16.372665,17.881809
2022-02-11 00:00:00+00:00,218.890945,28.610001,148.300262,194.876751,6.105549,16.351036,17.749066
2022-02-14 00:00:00+00:00,217.046494,28.370001,149.669464,183.896529,6.034888,16.091496,17.379293
2022-02-15 00:00:00+00:00,220.336594,30.070000,153.509171,192.377183,6.138970,16.043914,17.568925
2022-02-16 00:00:00+00:00,215.889969,29.230000,155.126434,193.470755,6.216316,16.065541,17.663736
...,...,...,...,...,...,...,...
2024-07-25 00:00:00+00:00,452.977875,17.610001,89.209999,175.929644,7.224044,14.827730,19.567832
2024-07-26 00:00:00+00:00,465.256165,17.900000,89.930000,178.384218,7.234891,14.773495,19.551560
2024-07-29 00:00:00+00:00,465.266144,17.969999,92.139999,175.075873,7.213197,14.806036,19.546138


In [25]:
tipo_cambio_colones = pd.read_excel("tipo_cambio_colones.xls")
tipo_cambio_colones

,FECHA,TIPO CAMBIO COMPRA,TIPO DE CAMBIO VENTA
0,2022-02-10,641.03,647.15
1,2022-02-11,640.19,646.93
2,2022-02-12,638.63,644.59
3,2022-02-13,638.63,644.59
4,2022-02-14,638.63,644.59
...,...,...,...
898,2024-07-27,523.18,527.73
899,2024-07-28,523.18,527.73
900,2024-07-29,523.18,527.73
901,2024-07-30,522.02,527.18


In [26]:
import pandas as pd

def eliminar_fines_de_semana(df, columna_fecha):
    # Asegúrate de que la columna es de tipo datetime
    df[columna_fecha] = pd.to_datetime(df[columna_fecha])

    # Filtra el DataFrame para excluir sábados (5) y domingos (6)
    df = df[~df[columna_fecha].dt.dayofweek.isin([5, 6])]

    return df

eliminar_fines_de_semana(tipo_cambio_colones,"FECHA")
tipo_cambio_colones

,FECHA,TIPO CAMBIO COMPRA,TIPO DE CAMBIO VENTA
0,2022-02-10,641.03,647.15
1,2022-02-11,640.19,646.93
2,2022-02-12,638.63,644.59
3,2022-02-13,638.63,644.59
4,2022-02-14,638.63,644.59
...,...,...,...
898,2024-07-27,523.18,527.73
899,2024-07-28,523.18,527.73
900,2024-07-29,523.18,527.73
901,2024-07-30,522.02,527.18


# Riesgo de precio

In [27]:
rendimientos = cartera_usd.diff().dropna()
rendimientos

,META,TRIP,DIS,IAG.L,ENEL.MI,ENG.MC,ELE.MC
Date,,,,,,,
2022-02-11 00:00:00+00:00,-8.494431,-0.730000,-2.668961,-4.084107,-0.036286,-0.021629,-0.132743
2022-02-14 00:00:00+00:00,-1.844452,-0.240000,1.369202,-10.980222,-0.070661,-0.259540,-0.369773
2022-02-15 00:00:00+00:00,3.290100,1.699999,3.839706,8.480654,0.104082,-0.047581,0.189632
2022-02-16 00:00:00+00:00,-4.446625,-0.840000,1.617264,1.093573,0.077347,0.021627,0.094811
2022-02-17 00:00:00+00:00,-8.803482,-0.730000,-3.373413,-7.900424,-0.026737,-0.112466,0.132737
...,...,...,...,...,...,...,...
2024-07-25 00:00:00+00:00,-7.852478,0.180000,-0.590004,1.654181,0.031456,0.065082,0.054236
2024-07-26 00:00:00+00:00,12.278290,0.289999,0.720001,2.454574,0.010847,-0.054235,-0.016272
2024-07-29 00:00:00+00:00,0.009979,0.070000,2.209999,-3.308345,-0.021694,0.032540,-0.005423


In [28]:
cartera_oceanica = pd.read_excel("Inversiones -07-2024     Oficial Riesgos F.xlsx")

In [29]:
acciones = cartera_oceanica[cartera_oceanica['[E] CodTitulo'] == 'AC'].reset_index()
acciones

,index,[E]CodReferencia,[E] CodISIN,[E]Ramo,[E] CodTitulo,[E] NemoEmisor,[E]NemotInstrumento,[E] FecAdquisicion,[E] FecVencimiento,[A] CodMoneda,...,[E]Calificadora,[E]PlazoCalificacion,[E]RiesgoEmisor,[E]RiesgoCreditoEmision,[E]CodCustodio,[E]NumCustodia,[E]ValorLibrosMonedaOriginal,[E]ValorLibrosColones,[E]Deterioro,[E]CodTipoValoracion
0,4,A13000000121,US30303M1027,G10,AC,FB,afb,2021-12-29,NaT,USD,...,MO,LP,A1,NaN,COTROS,14906.0,NaN,NaN,NaN,RI
1,5,A13000000122,US8969452015,G06,AC,TRIP,atrip,2021-12-28,NaT,USD,...,SP,LP,BB-,NaN,COTROS,14906.0,NaN,NaN,NaN,RI
2,6,A13000000124,ES0177542018,G10,AC,ICAG,aicag,2021-12-29,NaT,EUR,...,SP,LP,BB,NaN,COTROS,14906.0,NaN,NaN,NaN,RI
3,7,A13000000125,IT0003128367,G06,AC,ENEI,aenei,2021-12-30,NaT,EUR,...,SP,LP,BBB+,NaN,COTROS,14906.0,NaN,NaN,NaN,RI
4,8,A13000000126,ES0130960018,G06,AC,ENAG,aenag,2021-12-30,NaT,EUR,...,SP,LP,BBB,NaN,COTROS,14906.0,NaN,NaN,NaN,RI
5,10,A13000000129,ES0130670112,G06,AC,ELE,aele,2022-01-04,NaT,EUR,...,SP,LP,BBB+,NaN,COTROS,14906.0,NaN,NaN,NaN,RI
6,11,A13000000130,ES0177542018,G10,AC,ICAG,aicag,2022-01-11,NaT,EUR,...,SP,LP,BB,NaN,COTROS,14906.0,NaN,NaN,NaN,RI
7,12,A13000000131,US2546871060,G01,AC,WALT,awalt,2022-02-10,NaT,USD,...,SP,LP,BBB+,NaN,COTROS,14906.0,NaN,NaN,NaN,RI


In [30]:
montos = acciones['[E] CostoAdquisicion']
montos_numericos = pd.to_numeric(montos, errors='coerce')
vector_montos = np.array(montos_numericos)
aicag = vector_montos[2]+vector_montos[6]
vector_montos = np.delete(vector_montos, [2, 6])
vector_montos = np.insert(vector_montos, 3, aicag)

print(vector_montos)

[27427800.5739     26592340.6911     28435381.4957058  86356794.66485736
 23822222.17386501 29137830.60257182 58191584.208     ]


In [31]:
prom_tipo_cambio = np.mean([1.1318,1.1326])
vector_ordenado = np.array([vector_montos[0],vector_montos[1],vector_montos[6],vector_montos[3]*prom_tipo_cambio,vector_montos[2]*1.1302,vector_montos[4]*1.1302,vector_montos[5]*1.1281])
vector_ordenado

array([27427800.5739    , 26592340.6911    , 58191584.208     ,
       97773162.9195515 , 32137668.1664467 , 26923875.50090224,
       32870386.70276127])

In [67]:
import yfinance as yf
from datetime import datetime, timedelta

# Lista para almacenar los precios ajustados iniciales
precios_iniciales = []

for isin, fecha in isin_data.items():
    ticker = isin_to_ticker[isin]

    # Convertir la fecha a datetime
    fecha_dt = datetime.strptime(fecha, "%Y-%m-%d")

    # Sumar un día para obtener la fecha de fin
    fechas_iniciales = fecha_dt + timedelta(days=1)

    try:
        # Descargar datos desde Yahoo Finance
        data = yf.download(ticker, start=fecha_dt, end=fechas_iniciales)

        # Verificar si se obtuvo algún dato
        if not data.empty:
            precio_ajustado = data['Adj Close'].iloc[0]
            precios_iniciales.append(precio_ajustado)
        else:
            print(f"No se encontraron datos para {ticker} en {fecha}")

    except Exception as e:
        print(f"Error al descargar datos para {ticker}: {e}")

# Imprimir la lista de precios iniciales
precios_iniciales


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[Ticker
 META    341.910553
 Name: 2021-12-29 00:00:00+00:00, dtype: float64,
 Ticker
 TRIP    27.370001
 Name: 2021-12-28 00:00:00+00:00, dtype: float64,
 Ticker
 IAG.L    141.147995
 Name: 2021-12-29 00:00:00+00:00, dtype: float64,
 Ticker
 ENEL.MI    5.769054
 Name: 2021-12-30 00:00:00+00:00, dtype: float64,
 Ticker
 ENG.MC    15.561132
 Name: 2021-12-30 00:00:00+00:00, dtype: float64,
 Ticker
 ELE.MC    16.920288
 Name: 2022-01-04 00:00:00+00:00, dtype: float64,
 Ticker
 DIS    150.969223
 Name: 2022-02-10 00:00:00+00:00, dtype: float64]

In [79]:
precios_iniciales_df = pd.concat(precios_iniciales, axis=0)
precios_iniciales_df

,0
Ticker,
META,341.910553
TRIP,27.370001
IAG.L,141.147995
ENEL.MI,5.769054
ENG.MC,15.561132
ELE.MC,16.920288
DIS,150.969223


In [80]:
numero_acciones = vector_ordenado / precios_iniciales_df.values
numero_acciones

array([   80219.22790907,   971587.13466246,   412273.54458711,
       16947866.30923613,  2065252.53280162,  1591218.50432788,
         217729.05791448])

In [81]:
import yfinance as yf
from datetime import datetime, timedelta

# Lista para almacenar los precios ajustados iniciales
precios_finales = []

for isin, fecha in isin_data.items():
    ticker = isin_to_ticker[isin]

    try:
        # Descargar datos desde Yahoo Finance
        data = yf.download(ticker, start='2024-07-31', end='2024-08-01')

        # Verificar si se obtuvo algún dato
        if not data.empty:
            precio_ajustado = data['Adj Close'].iloc[0]
            precios_finales.append(precio_ajustado)
        else:
            print(f"No se encontraron datos para {ticker} en {fecha}")

    except Exception as e:
        print(f"Error al descargar datos para {ticker}: {e}")

# Imprimir la lista de precios iniciales
precios_finales


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[Ticker
 META    474.377441
 Name: 2024-07-31 00:00:00+00:00, dtype: float64,
 Ticker
 TRIP    17.629999
 Name: 2024-07-31 00:00:00+00:00, dtype: float64,
 Ticker
 IAG.L    163.12796
 Name: 2024-07-31 00:00:00+00:00, dtype: float64,
 Ticker
 ENEL.MI    6.594
 Name: 2024-07-31 00:00:00+00:00, dtype: float64,
 Ticker
 ENG.MC    13.84
 Name: 2024-07-31 00:00:00+00:00, dtype: float64,
 Ticker
 ELE.MC    17.9
 Name: 2024-07-31 00:00:00+00:00, dtype: float64,
 Ticker
 DIS    93.690002
 Name: 2024-07-31 00:00:00+00:00, dtype: float64]

In [82]:
precios_finales_df = pd.concat(precios_finales, axis=0)
precios_finales_df

,2024-07-31 00:00:00+00:00
Ticker,
META,474.377441
TRIP,17.629999
IAG.L,163.127960
ENEL.MI,6.594000
ENG.MC,13.840000
ELE.MC,17.900000
DIS,93.690002


In [83]:
valor_de_cartera = numero_acciones* precios_finales_df.values
valor_de_cartera

array([3.80541921e+07, 1.71290804e+07, 6.72533424e+07, 1.11754228e+08,
       2.85830954e+07, 2.84828106e+07, 2.03990360e+07])

Intento de ACP

In [33]:
from sklearn.preprocessing import StandardScaler

# Normalizar los datos
scaler = StandardScaler()
rendimientos_normalizados = scaler.fit_transform(rendimientos)

In [34]:
from sklearn.decomposition import PCA

# Configurar el modelo de PCA para extraer todas las componentes
pca = PCA(n_components=len(rendimientos.columns))
componentes_principales = pca.fit_transform(rendimientos_normalizados)
componentes_principales

array([[-1.87454625, -1.35897724, -0.16391682, ..., -0.1568373 ,
        -0.1002146 , -0.31562807],
       [-2.76321373,  0.85741872,  0.87366123, ...,  0.46941996,
        -1.24250636, -0.5007202 ],
       [ 3.13512051,  2.16474673, -1.49591482, ...,  0.15759674,
        -0.24026438,  0.20397951],
       ...,
       [-0.09395021,  0.42354826,  0.39187261, ...,  0.52945335,
        -0.14979378,  0.09652401],
       [ 0.33922375,  0.29906309, -0.38636574, ...,  0.61322698,
         0.50978084,  0.17265388],
       [ 0.01822711,  0.35558296,  1.38334026, ..., -0.10110455,
         1.07910481, -0.15274357]])

In [35]:
covar = np.cov(np.transpose(componentes_principales)) #valores propios de los componentes principales en la diagonal
covar

array([[ 2.63156800e+00,  2.04472730e-16, -6.26197735e-16,
         2.36421594e-16, -9.58465921e-17, -1.85303411e-16,
        -2.87539776e-16],
       [ 2.04472730e-16,  1.59125464e+00, -2.81150003e-16,
         4.15335232e-17, -3.83386368e-17,  4.95207392e-17,
         3.13098867e-16],
       [-6.26197735e-16, -2.81150003e-16,  8.09273628e-01,
         8.94568193e-17, -1.91693184e-17,  2.20447162e-16,
         3.99360800e-17],
       [ 2.36421594e-16,  4.15335232e-17,  8.94568193e-17,
         6.75015114e-01, -3.13098867e-16, -3.83386368e-17,
        -6.86900576e-17],
       [-9.58465921e-17, -3.83386368e-17, -1.91693184e-17,
        -3.13098867e-16,  5.63944823e-01, -9.90414785e-17,
        -9.90414785e-17],
       [-1.85303411e-16,  4.95207392e-17,  2.20447162e-16,
        -3.83386368e-17, -9.90414785e-17,  4.29142236e-01,
         3.69009379e-16],
       [-2.87539776e-16,  3.13098867e-16,  3.99360800e-17,
        -6.86900576e-17, -9.90414785e-17,  3.69009379e-16,
         3.1239148

In [36]:
sigma = np.sqrt(np.dot(np.dot(np.transpose(vector_ordenado),covar),vector_ordenado))
print(sigma)

116315502.9743876


In [37]:
from scipy.stats import norm

nivel_conf = 0.95
zq = norm.ppf(nivel_conf)
zq

1.6448536269514722

In [38]:
VaR = sigma*zq
VaR

191321976.9381062